In [1]:
import networkx as nx
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
import logging
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.wkt import loads
from scipy.spatial.distance import cdist

import plotly.graph_objects as go

# Let's brag about digital twins ;)
import sgp_dt as dt

%load_ext autoreload # to play with the lib in real time
%autoreload 2

# Checking what we have in the network

In [21]:
# For reference only, the geodataframe with all path
fname = '../data/s3.zip'
SLC = gpd.read_file(fname)
SLC

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,...,Lamps25m,Lamps10m,CCTV100mRE,CCTV50mRE,CCTV25mRE,Length,Trees50m,Trees25m,Trees10m,geometry
0,1,5115,tertiary,Keng Lee Road,None,F,50,0,F,F,...,6,4,0,0,0,100.099656,120,89,50,"LINESTRING (103.84379 1.31370, 103.84468 1.31383)"
1,2,5115,tertiary,Keng Lee Road,None,F,50,0,F,F,...,4,2,0,0,0,38.075346,51,29,18,"LINESTRING (103.84468 1.31383, 103.84495 1.313..."
2,3,5153,footway,None,None,B,0,0,F,F,...,1,0,0,0,0,99.549892,36,12,6,"LINESTRING (103.84083 1.31463, 103.84115 1.31379)"
3,4,5153,footway,None,None,B,0,0,F,F,...,2,0,0,0,0,47.704451,57,19,7,"LINESTRING (103.84115 1.31379, 103.84126 1.313..."
4,5,5122,residential,Sarkies Road,None,B,50,0,F,F,...,4,1,0,0,0,21.937559,46,17,1,"LINESTRING (103.83721 1.31461, 103.83727 1.314..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50343,50344,5153,footway,None,None,B,0,0,F,F,...,2,1,0,0,0,100.068655,42,24,10,"LINESTRING (103.84753 1.29756, 103.84767 1.297..."
50344,50345,5153,footway,None,None,B,0,0,F,F,...,0,0,0,0,0,36.577630,109,5,1,"LINESTRING (103.84720 1.29759, 103.84721 1.297..."
50345,50346,5153,footway,None,None,B,0,0,F,F,...,0,0,0,0,0,6.727414,104,4,2,"LINESTRING (103.84721 1.29759, 103.84721 1.297..."
50346,50347,5153,footway,None,None,B,0,0,F,F,...,0,0,0,0,0,3.673968,92,5,1,"LINESTRING (103.84721 1.29753, 103.84720 1.29750)"


#### With some stats

In [24]:
stats = SLC.groupby('fclass')['Length'].agg(['sum','count']).astype(int).reset_index()
stats.columns = ["network","length (m)","number of segments"]
stats

,network,length (m),number of segments
0,bridleway,880,11
1,cycleway,46307,698
2,footway,800191,16713
3,living_street,2090,65
4,path,16906,284
5,pedestrian,70678,1469
6,residential,312628,10205
7,secondary,74031,1973
8,secondary_link,2608,73
9,service,545996,15409


# Starting the streamlit

In [2]:
def loadShp(path):
    # Reading the overall network
    G= nx.readwrite.nx_shp.read_shp(path)
    # Support to network identification
    df_nodes = pd.read_csv("data/SG_nodes.txt", index_col=0)
    # Some cleaning necessary for csv loadup
    dfNodes = pd.read_csv("data/dfNodes.csv.zip")
    dfNodes.Pos = dfNodes.Pos.apply(lambda x: eval(x))
    dfEdges = pd.read_csv("data/dfEdges.csv.zip") 
    dfEdges.geometry = dfEdges.geometry.apply(lambda x: loads(x.replace('\'', '')))
    # Additional layers  
    gTrees = gpd.read_file('data/sTrees.zip') 
    gLamps = gpd.read_file('data/sLamps.zip') 
    gPark = gpd.read_file('data/sParks.zip') 
    gCCTV = gpd.read_file('data/sCCTV.zip') 
    return G, df_nodes, dfNodes, dfEdges, gTrees, gLamps, gPark, gCCTV

# This call should be cached
G, df_nodes, dfNodes, dfEdges, gTrees, gLamps, gPark, gCCTV = loadShp("data/s3/StreetsLampsCCTVTrees.shp")

In [3]:
security   = 1 
cctv_perf  = 5.0
lamps_perf = 5.0
trees_perf = 5.0
#Different values for different types for paths
footway     = 2.0
residential = 1.5 
steps       = 2.0

UseDummyPoints = True

if UseDummyPoints:
    start,end = dt.getStartEnd("nop", "end", df_nodes, dummy= UseDummyPoints)
else:
    start,end = dt.getStartEnd(start_point, end_point, df_nodes, dummy= UseDummyPoints)

In [13]:
weighted_G, pos, labels = dt.get_weighted_graph(G,security=1,cctv_perf=cctv_perf,lamps_perf=lamps_perf)

In [14]:
dt.mapIt(start,end,weighted_G,dfNodes,dfEdges)

In [30]:
def get_weighted_graphTrees(G,security=1,cctv_perf=1.0,lamps_perf=1.0,trees_perf=1.0):
    """
    Takes a graph G as input and adds the weights
    """
    weighted_G = nx.Graph()
    for data in G.edges(data=True):
        data=data
        coord1 = data[0]
        coord2 = data[1]

        # ZE formula to tweak
        data[2]['weight'] = data[2]['Length']
        data[2]['weight'] = data[2]['weight'] / (1.0 + 0.1*data[2]['CCTV50mRE'] * cctv_perf )
        data[2]['weight'] = data[2]['weight'] / (1.0 + 0.1*data[2]['Lamps50m']  * lamps_perf) 
        data[2]['weight'] = data[2]['weight'] / (1.0 + 0.1*data[2]['Trees10m']  * trees_perf) 

        # It is adapted for pedestrians ?
        relativeEase = 1.0
        if data[2]['fclass'] == "pedestrian":
            relativeEase = 0.3
        elif data[2]['fclass'] == "footway":
            relativeEase = 0.3
        elif data[2]['fclass'] == "steps":
            relativeEase = 0.6            
        data[2]['weight'] = data[2]['weight'] * relativeEase

        weighted_G.add_edge(coord1,coord2,weight=data[2]['weight'])

    return weighted_G

security   = 1.0
cctv_perf  = 1.0
lamps_perf = 1.0
trees_perf = 1.0

weighted_G = get_weighted_graphTrees(G,security=1,cctv_perf=cctv_perf,lamps_perf=lamps_perf,trees_perf=trees_perf)

In [31]:
dt.mapIt(start,end,weighted_G,dfNodes,dfEdges)